# Kipoi python-sdk

## Quick start

There are three basic building blocks in kipoi:
- **Source** - provides Models and DataLoaders.
- **Model** - makes the prediction given the numpy arrays. 
- **Dataloader** - loads the data from raw files and transforms them into a form that is directly consumable by the Model

![img](../docs/img/kipoi-workflow.png)

## List of main commands


- `kipoi.list_sources()`
- `kipoi.get_source()`


- `kipoi.list_models()`
- `kipoi.list_dataloaders()`


- `kipoi.get_model()`
- `kipoi.get_dataloader_factory()`



### Source

Available sources are specified in the config file located at: `~/.kipoi/config.yaml`. Here is an example config file:

```yaml
model_sources:
    kipoi: # default
        type: git-lfs # git repository with large file storage (git-lfs)
        remote_url: git@github.com:kipoi/models.git # git remote
        local_path: ~/.kipoi/models/ # local storage path
    gl:
        type: git-lfs  # custom model
        remote_url: https://i12g-gagneurweb.informatik.tu-muenchen.de/gitlab/gagneurlab/model-zoo.git
        local_path: /s/project/model-zoo
```

There are three different model sources possible: 
- **`git-lfs`** - git repository with source files tracked normally by git and all the binary files like model weights (located in `files*` directories) are tracked by [git-lfs](https://git-lfs.github.com). 
  - Requires `git-lfs` to be installed.
- **`git`** - all the files including weights (not recommended)
- **`local`** - local directory containing models defined in subdirectories

For **`git-lfs`** source type, larger files tracked by `git-lfs` will be downloaded into the specified directory `local_path` only after the model has been requested (when invoking `kipoi.get_model()`).

#### Note

A particular model/dataloader is defined by its source (say `kipoi` or `my_git_models`) and the relative path of the desired model directory from the model source root (say `rbp/`).

A directory is considered a model if it contains a `model.yaml` file.

In [2]:
import kipoi

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
kipoi.list_sources()

,source,type,location,local_size,n_models,n_dataloaders
0,kipoi,git-lfs,/data/ouga/home/ag_ga...,11G,173,173
1,gl,git,/s/project/model-zoo/,167M,1,1


In [5]:
s = kipoi.get_source("kipoi")

In [6]:
s

GitLFSSource(remote_url='git@github.com:kipoi/models.git', local_path='/data/ouga/home/ag_gagneur/avsec/.kipoi/models/')

In [7]:
kipoi.list_models()

,source,model,version,authors,doc,type,inputs,targets,license,cite_as,trained_on,training_procedure,tags
0,kipoi,MaxEntScan,0.1,[Author(name='Gene Ye...,MaxEnt Splicing Model...,custom,seq,psi,MIT,https://doi.org/10.10...,Described in http://o...,Described in http://o...,[]
1,kipoi,extended_coda,0.1,[Author(name='Johnny ...,Single bp resolution ...,keras,[H3K27AC_subsampled],[H3K27ac],MIT,https://doi.org/10.10...,Described in https://...,Described in https://...,[]
2,kipoi,DeepSEA,0.1,[Author(name='Lara Ur...,This CNN is based on ...,keras,seq,TFBS_DHS_probs,MIT,https://doi.org/10.10...,ENCODE and Roadmap Ep...,https://www.nature.co...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,kipoi,rbp_eclip/HNRNPK,0.1,[Author(name='Ziga Av...,'RBP binding model fr...,keras,"[seq, dist_tss, dist_...",None,MIT,https://doi.org/10.10...,RBP occupancy peaks m...,Single task training ...,[]
173,kipoi,rbp_eclip/FXR2,0.1,[Author(name='Ziga Av...,'RBP binding model fr...,keras,"[seq, dist_tss, dist_...",None,MIT,https://doi.org/10.10...,RBP occupancy peaks m...,Single task training ...,[]
174,kipoi,rbp_eclip/GRSF1,0.1,[Author(name='Ziga Av...,'RBP binding model fr...,keras,"[seq, dist_tss, dist_...",None,MIT,https://doi.org/10.10...,RBP occupancy peaks m...,Single task training ...,[]


## Model

Let's choose to use the HAL model from kipoi

In [8]:
# Note. Install all the dependencies for that model using
# kipoi install 
model = kipoi.get_model("rbp_eclip/UPF1")

Using TensorFlow backend.
2018-01-25 16:23:06,774 [INFO] successfully loaded the dataloader from /data/ouga/home/ag_gagneur/avsec/.kipoi/models/rbp_eclip/UPF1/dataloader.py::SeqDistDataset
2018-01-25 16:23:08,836 [INFO] successfully loaded the model from model_files/model.h5
2018-01-25 16:23:08,836 [INFO] dataloader.output_schema is compatible with model.schema


### Available fields:

#### Model

- type
- args
- info
  - authors
  - name
  - version
  - tags
  - doc
- schema
  - inputs
  - targets
- default_dataloader - loaded dataloader class


- predict_on_batch()
- source
- source_dir
- pipeline
  - predict()
  - predict_example()
  - predict_generator()
  
#### Dataloader

- type
- defined_as
- args
- info (same as for the model)
- output_schema
  - inputs
  - targets
  - metadata


- source
- source_dir
- example_kwargs
- init_example()
- batch_iter()
- batch_train_iter()
- batch_predict_iter()
- load_all()

In [9]:
model

In [10]:
model.type

'keras'

### Info

In [11]:
model.info

ModelInfo(authors=[Author(name='Ziga Avsec', github='avsecz', email=None)], doc='\'RBP binding model from Avsec et al: "Modeling positional effects of regulatory sequences with spline transformations increases prediction accuracy of deep neural networks". \'\n', name=None, version='0.1', license='MIT', tags=[], cite_as='https://doi.org/10.1093/bioinformatics/btx727', trained_on='RBP occupancy peaks measured by eCLIP-seq (Van Nostrand et al., 2016 - https://doi.org/10.1038/nmeth.3810), https://github.com/gagneurlab/Manuscript_Avsec_Bioinformatics_2017\n', training_procedure='Single task training with ADAM')

In [12]:
model.info.version

'0.1'

### Schema

In [13]:
model.schema.inputs

OrderedDict([('seq',
              ArraySchema(shape=(101, 4), doc='One-hot encoded RNA sequence', name='seq', special_type=<ArraySpecialType.DNASeq: 'DNASeq'>, associated_metadata=[], column_labels=None)),
             ('dist_tss',
              ArraySchema(shape=(1, 10), doc='Distance the nearest TSS site transformed with B-splines', name='dist_tss', special_type=None, associated_metadata=[], column_labels=None)),
             ('dist_polya',
              ArraySchema(shape=(1, 10), doc='Distance the nearest Poly-A site transformed with B-splines', name='dist_polya', special_type=None, associated_metadata=[], column_labels=None)),
             ('dist_exon_intron',
              ArraySchema(shape=(1, 10), doc='Distance the nearest exon_intron (splice donor) site transformed with B-splines', name='dist_exon_intron', special_type=None, associated_metadata=[], column_labels=None)),
             ('dist_intron_exon',
              ArraySchema(shape=(1, 10), doc='Distance the nearest intron_

In [14]:
model.schema.targets

ArraySchema(shape=(1,), doc='Predicted binding strength', name=None, special_type=None, associated_metadata=[], column_labels=None)

### Default dataloader

Model already has the default dataloder present. To use it, specify

In [15]:
model.source_dir

'/data/ouga/home/ag_gagneur/avsec/.kipoi/models/rbp_eclip/UPF1'

In [16]:
model.default_dataloader

dataloader.SeqDistDataset

In [17]:
model.default_dataloader.info

Info(authors=[Author(name='Ziga Avsec', github='avsecz', email=None)], doc='RBP binding prediction for UPF1 protein', name=None, version='0.1', license='MIT', tags=[])

### Predict_on_batch

In [18]:
model.predict_on_batch

<bound method KerasModel.predict_on_batch of <kipoi.model.KerasModel object at 0x7f7bce92fdd8>>

### Pipeline

Pipeline object will take the dataloader arguments and run the whole pipeline:

```
dataloader arguments --Dataloader-->  numpy arrays --Model--> prediction
```

In [19]:
?model.pipeline.predict

Signature: model.pipeline.predict(dataloader_kwargs, batch_size=32)
Docstring:
# Arguments
    preproc_kwargs: Keyword arguments passed to the pre-processor

:return: Predict the whole array
File:      /data/nasif12/home_if12/avsec/projects-work/kipoi/kipoi/pipeline.py
Type:      method


In [20]:
?model.pipeline.predict_generator

Signature: model.pipeline.predict_generator(dataloader_kwargs, batch_size=32)
Docstring:
Prediction generator

# Arguments
    preproc_kwargs: Keyword arguments passed to the pre-processor

# Yields
    model batch prediction
File:      /data/nasif12/home_if12/avsec/projects-work/kipoi/kipoi/pipeline.py
Type:      method


### Others

In [21]:
# Model source
model.source

GitLFSSource(remote_url='git@github.com:kipoi/models.git', local_path='/data/ouga/home/ag_gagneur/avsec/.kipoi/models/')

In [22]:
# model location directory
model.source_dir

'/data/ouga/home/ag_gagneur/avsec/.kipoi/models/rbp_eclip/UPF1'

## DataLoader

In [23]:
DataLoader = kipoi.get_dataloader_factory("rbp_eclip/UPF1")

2018-01-25 16:23:26,782 [INFO] git-lfs pull -I rbp_eclip/UPF1/**
2018-01-25 16:23:27,028 [INFO] dataloader rbp_eclip/UPF1 loaded
2018-01-25 16:23:27,050 [INFO] successfully loaded the dataloader from /data/ouga/home/ag_gagneur/avsec/.kipoi/models/rbp_eclip/UPF1/dataloader.py::SeqDistDataset


In [24]:
?DataLoader

Init signature: DataLoader(intervals_file, fasta_file, gtf_file, target_file=None, use_linecache=False)
Docstring:     
Args:
    intervals_file: file path; tsv file
        Assumes bed-like `chrom start end id score strand` format.
    fasta_file: file path; Genome sequence
    gtf_file: file path; Genome annotation GTF file.
    preproc_transformer: file path; tranformer used for pre-processing.
    target_file: file path; path to the targets
    batch_size: int
Type:           type


## Run dataloader on some examples

In [25]:
# each dataloader already provides the example files
DataLoader.example_kwargs

{'fasta_file': 'example_files/hg38_chr22.fa',
 'gtf_file': 'example_files/gencode.v24.annotation_chr22.gtf',
 'intervals_file': 'example_files/intervals.bed',
 'target_file': 'example_files/targets.tsv'}

In [26]:
import os

In [27]:
# cd into the source directory 
os.chdir(DataLoader.source_dir)

In [28]:
!tree

.
├── custom_keras_objects.py -> ../template/custom_keras_objects.py
├── dataloader_files
│   └── position_transformer.pkl
├── dataloader.py -> ../template/dataloader.py
├── dataloader.yaml -> ../template/dataloader.yaml
├── example_files -> ../template/example_files
├── model_files
│   └── model.h5
├── model.yaml -> ../template/model.yaml
└── __pycache__
    ├── custom_keras_objects.cpython-35.pyc
    └── dataloader.cpython-35.pyc

4 directories, 8 files


In [29]:
dl = DataLoader(**DataLoader.example_kwargs)
# could be also done with DataLoader.init_example()

2018-01-25 16:23:31,080 [INFO] Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'havana_gene', 'ont', 'havana_transcript', 'protein_id', 'ccdsid']
INFO:2018-01-25 16:23:31,173:genomelake] Running landmark extractors..
2018-01-25 16:23:31,173 [INFO] Running landmark extractors..
INFO:2018-01-25 16:23:31,407:genomelake] Done!
2018-01-25 16:23:31,407 [INFO] Done!


In [30]:
# This particular dataloader is of type Dataset
# i.e. it implements the __getitem__ method:
dl[0].keys()

dict_keys(['metadata', 'inputs', 'targets'])

In [31]:
dl[0]["inputs"].keys()

dict_keys(['dist_start_codon', 'dist_intron_exon', 'dist_polya', 'dist_tss', 'dist_stop_codon', 'dist_gene_end', 'dist_gene_start', 'seq', 'dist_exon_intron'])

In [32]:
dl[0]["inputs"]["seq"][:5]

array([[ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25]], dtype=float32)

In [33]:
len(dl)

14

### Get the whole dataset

In [34]:
whole_data = dl.load_all()

100%|██████████| 1/1 [00:00<00:00,  6.62it/s]


In [35]:
whole_data.keys()

dict_keys(['metadata', 'inputs', 'targets'])

In [36]:
whole_data["inputs"]["seq"].shape

(14, 101, 4)

### Get the iterator to run predictions

In [37]:
it = dl.batch_iter(batch_size=1, shuffle=False, num_workers=0, drop_last=False)

In [38]:
next(it)["inputs"]["seq"].shape

(1, 101, 4)

In [39]:
model.predict_on_batch(next(it)["inputs"])

array([[ 0.0005]], dtype=float32)

### Train the Keras model

Keras model is stored under the `.model` attribute.

In [40]:
model.model.compile("adam", "binary_crossentropy")

In [41]:
from itertools import cycle

In [42]:
train_it = cycle(dl.batch_train_iter(batch_size=2))

**NOTE**: In order to use the iterator with Keras, we have to wrap the returned
iterator with `itertools.cycle` (i.e. run `itertools.cycle(dl.batch_train_iter(...))`)
in order to cycle through the dataset indefinitely,

In [51]:
# model.model.summary()

In [44]:
model.model.fit_generator(train_it, steps_per_epoch=3, epochs=1)

Epoch 1/1
3/3 [==============================] - 1s 172ms/step - loss: 1.0374


## Pipeline: `raw files -[dataloader]-> numpy arrays -[model]-> prediction`

In [45]:
example_kwargs = model.default_dataloader.example_kwargs

In [46]:
model.pipeline.predict(example_kwargs)

2018-01-25 16:23:41,248 [INFO] Initialized data generator. Running batches...
2018-01-25 16:23:42,548 [INFO] Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'havana_gene', 'ont', 'havana_transcript', 'protein_id', 'ccdsid']
INFO:2018-01-25 16:23:42,616:genomelake] Running landmark extractors..
2018-01-25 16:23:42,616 [INFO] Running landmark extractors..
INFO:2018-01-25 16:23:42,853:genomelake] Done!
2018-01-25 16:23:42,853 [INFO] Done!


array([[ 0.143 ],
       [ 0.0005],
       [ 0.0005],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.0005],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.143 ],
       [ 0.143 ]], dtype=float32)

In [47]:
next(model.pipeline.predict_generator(example_kwargs, batch_size=2))

2018-01-25 16:23:43,706 [INFO] Initialized data generator. Running batches...
2018-01-25 16:23:45,006 [INFO] Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'havana_gene', 'ont', 'havana_transcript', 'protein_id', 'ccdsid']
INFO:2018-01-25 16:23:45,080:genomelake] Running landmark extractors..
2018-01-25 16:23:45,080 [INFO] Running landmark extractors..
INFO:2018-01-25 16:23:45,225:genomelake] Done!
2018-01-25 16:23:45,225 [INFO] Done!


array([[ 0.143 ],
       [ 0.0005]], dtype=float32)